In [2]:
from music21 import *
import pandas as pd
import numpy as np
import music21

In [3]:
class AudioAnalysis:

    '''
    This class takes a pandas DataFrame representing the correct and incorrect notes respectively played by a solo instrument
    and converts it into a music21 score that highlights any mistakes in intonation, ignoring issues in time.
    '''
    
    def __init__(self, correct_df: pd.DataFrame, input_df: pd.DataFrame, score: str):
        self.correct_df = correct_df
        self.input_df = input_df
        self.score = converter.parse(score)

    def compare_dataframe(self) -> pd.DataFrame:
        correct_notes = self.correct_df['Note Name']
        input_notes = self.input_df['Note Name']
        is_correct = []
        for i in range(len(correct_notes)):
            if correct_notes[i] == input_notes[i]:
                is_correct.append(True)
            else:
                is_correct.append(False)
        new_df = self.correct_df
        new_df.insert(3, "Played Notes", input_notes)
        new_df.insert(4, "Correct", is_correct)
        return new_df
    
    def generate_overlay_score(self):
        df = self.compare_dataframe()
        # correct_score = stream.Stream()
        # input_score = stream.Stream()
        new_score = stream.Stream()
        correct_notes = df['Note Name']
        # note_type = df['Note Type']
        is_correct = df["Correct"]
        input_notes = self.input_df['Played Notes']
        time_signature = self.score.getTimeSignatures()[0].ratioString
        new_score.append(meter.TimeSignature(time_signature))
        for i in range(len(is_correct)):
            if is_correct[i]:
                new_note = music21.note.Note(correct_notes[i])
                new_note.duration.type = 'quarter'
                new_score.append(new_note)
            else:
                correct_note = music21.note.Note(correct_notes[i])
                incorrect_note = music21.note.Note(input_notes[i])
                correct_note.duration.type = 'quarter'
                incorrect_note.duration.type = 'quarter'
                incorrect_note.style.color = 'red'
                combined_chord = music21.chord.Chord([correct_note, incorrect_note])
                new_score.append(combined_chord)
        # correct_score.append(meter.TimeSignature(time_signature))
        # input_score.append(meter.TimeSignature(time_signature))

        # for note in correct_notes:
        #     correct_score.append(music21.note.Note(note))
        # for i in range(len(input_notes)):
        #     current_note = music21.note.Note(input_notes[i])
        #     current_note.duration.type = note_type[i]
        #     if is_correct[i]:
        #         input_score.append(current_note)
        #     else:
        #         current_note.style.color = 'red'
        #         input_score.append(current_note)
        return new_score.write('midi')
        
        
        
        
